In [ ]:
import tensorflow as tf
import numpy as np



def mfcc(audio):
    #maybe its better to put variables some place else
    sampling_rate = 16000
    frame_length = int(0.04 * sampling_rate)
    frame_step = int(0.02 * sampling_rate)
    num_mel_bins = 40
    lower_freq = 20
    upper_freq = 4000
    coefficients = 10
    num_spectrogram_bins = frame_length // 2 + 1
    seed = 42
    tf.random.set_seed(seed)
    np.random.seed(seed)

    tf_audio, rate = tf.audio.decode_wav(audio)
    tf_audio = tf.squeeze(tf_audio, 1)
    zero_padding = tf.zeros([sampling_rate] - tf.shape(audio), dtype=tf.float32)
    audio = tf.concat([audio, zero_padding], 0)
    audio.set_shape([sampling_rate])
    stft = tf.signal.stft(tf_audio, frame_length, frame_step, fft_length=frame_length)
    spectrogram = tf.abs(stft)
    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins, num_spectrogram_bins, sampling_rate, lower_freq, upper_freq)
    mel_spectrogram = tf.tensordot(spectrogram, linear_to_mel_weight_matrix, 1)
    log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
    mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
    mfccs = mfccs[..., :coefficients]
    mfccs = tf.expand_dims(mfccs, -1)
    mfccs = tf.expand_dims(mfccs, 0)

    return mfccs

In [ ]:
import json
import cherrypy
from SignalGenerator import mfcc

class SLowService:
    def POST(self):
        body = cherrypy.request.body.read()
        body = json.loads(body)
        print(body)
        audio = body.get('Audio')
        audio = base64.b64decode(audio)
        model_path = "../Prerequisite/kws_dscnn_True.tflite"
        interpreter = tf.lite.Interpreter(model_path=model_path)
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        interpreter.set_tensor(input_details[0]['index'], mfcc(audio))
        interpreter.invoke()
        predict_result = interpreter.get_tensor(output_details[0]['index'])
        predicted_label = np.argmax(predict_result)
        msg={'predicted_label':predicted_label}
        return json.dumps(msg)


if __name__ == '__main__':
    # conf probably needs modification
    conf = {
        '/': {
            'request.dispatch': cherrypy.dispatch.MethodDispatcher(),
            'tools.sessions.on': True,
        }
    }

    cherrypy.tree.mount(SLowService(),'/predict',conf)

    # To start cherrypy engine
    cherrypy.engine.start()
    cherrypy.engine.block()